# Análisis de Series de Tiempo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data as pdr
from datetime import datetime
from sklearn.linear_model import LinearRegression
pd.plotting.register_matplotlib_converters()
yf.pdr_override()

## ¿Qué es una serie de tiempo?

Una serie de tiempo es una secuencia de observaciones sobre intervalos de tiempo separados de manera regular. Por ejemplo:

- Las tasas mensuales de desempleo durante los cinco años previos
- La producción diaria en una planta de manufactura durante un mes
- La población década por década de un estado en el siglo anterior
- El precio de un activo durante siete años



Primero se declaran los activos a analizar, en este caso será sólo 1, además de indicar fecha de inicio y fecha final:

In [ ]:
assets = ["FB"]
start_date = "2013-01-01"
end_date = "2021-01-01"

Se obtienen los precios de cierre de Yahoo Finanzas y se acomodan en el DataFrame

In [ ]:
df=pd.DataFrame()
for asset in assets:
    df_asset = pdr.get_data_yahoo(asset, start=start_date, end=end_date)["Adj Close"]#El activo se conecta a Yahoo y hace lectura
    df_asset = df_asset.to_frame(name=asset) #Descarga la información
    df = pd.concat([df, df_asset], axis=1, sort=False) #al data frame se le agrega el nuevo activo

In [ ]:
df.head()

In [ ]:
df.reset_index(inplace = True)
df.rename(columns = {'index': 'date','Date':'date'}, inplace = True)
df.head()

In [ ]:
df.info()

In [ ]:
#Visualización
plt.plot(df['date'], df['FB'])
plt.title("Precio de cierre de FB. ")
plt.ylabel("Precio")
plt.xlabel("Fecha")
plt.savefig("img/Precio de cierre de FB.jpg")
plt.show()

In [ ]:
#¿Que pasa si los datos perdieron su orden?
df1 = df.copy()
df1 = df1.sample(frac=1).reset_index(drop=True)
df1.head()

In [ ]:
plt.plot(df1['date'], df1['FB'])
plt.title("Precio de cierre de FB.")
plt.ylabel("Precio")
plt.xlabel("Fecha")
plt.savefig("img/Precio de cierre de FB desordenado.jpg")
plt.show()

## Componentes de la Serie de Tiempo

Una serie de tiempo puede descomponerse en 4 componentes principales:
- Nivel (level)
- Tendencia (trend)
- Estacionalidad (seasonality)
- Ruido (noise)

Estas componentes pueden ser combinadas de forma aditiva y se definen de la siguiente forma:

\begin{align}
y(t) = \mathrm{nivel} + \mathrm{tendencia} + \mathrm{estacionalidad} + \mathrm{ruido}
\end{align}

### Nivel

El nivel de una serie temporal corresponde al **promedio aritmético** de todos los datos.

El nivel corresponde a un estadístico de los datos que nos da una idea de la escala en donde nos encontramos trabajando.

Aunque esta es en si una componente de la serie de tiempo, no utiliza la propiedad del orden temporal en ningún punto (obtenemos el mismo promedio independienmente de como ordenemos los datos).

Con esto podemos ver que aunque sí estamos extrayendo información de la serie de tiempo, no estamos haciendo uso del orden intrínseco de esta.

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#Vamos a observar como se ve el promedio de nuestros datos
plt.plot(df['date'], df['FB'], label = 'Stock Price')
plt.plot([min(df.date), max(df.date)], [df['FB'].mean(), df.iloc[:,1].mean()], label = 'Level')
plt.xlim([min(df.date), max(df.date)])
plt.title("Precio de acción con nivel.")
plt.ylabel("Precio")
plt.xlabel("Fecha")
plt.legend()
plt.savefig("img/Precio con promedio.jpg")
plt.show()

In [ ]:
#El promedio no depende del tiempo
plt.plot(df1['date'], df1['FB'], label = 'Stock Price')
plt.plot([min(df1.date), max(df1.date)], [df1['FB'].mean(), df1.iloc[:,1].mean()], label = 'Level')
plt.xlim([min(df1.date), max(df1.date)])
plt.title("Precio de acción a inicio de semana. ")
plt.ylabel("Precio")
plt.xlabel("Fecha")
plt.legend()
plt.savefig("img/Precio con promedio desordenado.jpg")
plt.show()

El promedio es una constante, no varía con respecto al tiempo.

In [ ]:
#Ahora... que pasa si removemos el nivel de los datos?
plt.plot(df.date, df['FB'] - df['FB'].mean())
plt.xlim([min(df.date), max(df.date)])
plt.title("Precio de acción sin nivel.")
plt.ylabel("Precio")
plt.xlabel("Fecha")
plt.savefig("img/Precio sin promedio.jpg")
plt.show()

Conclusión: Aunque el nivel o promedio aritmetico es una propiedad de la serie de tiempo, el hecho de quitarlo no cambia la naturaleza de nuestros datos. Esto es muy parecido a la normalización de los datos pero no se está realizando la división de la desviación estándar de los datos.

### Tendencia

Es una propiedad que, a diferencia del nivel, si describe la evolución global de nuestra variable a través del tiempo; es decir, define si la variable tiende a incrementar o a reducir su cantidad a través del tiempo (idea general, no específica).

Aunque la estacionalidad y el ruido pueden cambiar el comportamiento de la serie de forma local (en tiempos chicos), la tendencia es una métrica global de la serie, por lo que al analizarla en periodos largos de tiempo, se puede conocer la tendencia de los datos.

Nota: Puede cambiar debido a fenómenos externos y dependiendo del rango de fechas que estemos estudiando.

#### Regresión lineal, modelo simple para modelar la tendencia.

Es el modelo más sencillo para modelar la tendencia, ya que sólo necesita determinar 2 parámetros para poder realizar predicciones. Asumiendo que el precio de la acción es y(t), podemos proponer un modelo lineal simple de la siguiente forma:

\begin{align}
y(t) = at + b
\end{align}

Donde:
- a y b son constantes por definir
- t es el tiempo que se va a pronosticar.

Es decir, la única dependencia que tiene el precio de una acción es del tiempo.

In [ ]:
#Primero convertir los datos a una x equivalente. Datetime no es tan compatible con sklearn
dummy = np.linspace(0,df.shape[0] - 1, df.shape[0]).reshape(-1,1)

#Entrena objeto regresor
reg = LinearRegression().fit(dummy, df['FB'])

In [ ]:
#Realiza predicciones y evalua el modelo
results = reg.predict(dummy)
print(reg.score(dummy, df['FB'])) #Es la R2
print('El valor de R2 es muy alto')

In [ ]:
#Graficamos la data
plt.plot(df['date'], df['FB'])
plt.plot(df['date'], results)
plt.title("Precio de acción con tendencia.")
plt.ylabel("Precio")
plt.xlabel("Fecha")
plt.savefig("img/Precio con tendencia.jpg")
plt.show()

Se observa que el modelo describe el comportamiento global de los datos, pues este sigue un comportamiento creciente muy similar al que presentan los datos.

Se utilizaron todos los datos para entrenar al modelo, por lo que **no** se tiene una forma de evaluarlo.
Para crear un set de entrenamiento y un set de prueba se necesita escoger una fecha de corte. Todos los datos antes de la fecha son de entrenamiento y el resto son de prueba.

Esto es técnicamente un pronóstico, dado que estamos modelando la tendencia de nuestros datos en el futuro, utilizando datos de entrenamiento del pasado.

In [ ]:
trial_df = df[['date', 'FB']]
trial_df.shape #Cantidad de registros en el df

In [ ]:
test_sample = 350 #Declaro que voy a probar con 365 dias
train_df = trial_df.iloc[:(trial_df.shape[0] - test_sample),:] #Creo un set de entrenamiento de 0 hasta test_sample
test_df = trial_df.iloc[(trial_df.shape[0] - test_sample):,:] #creo un set de prueba con valores de test sample

In [ ]:
print(train_df.shape, test_df.shape) #Checo mis registros que estén bien

In [ ]:
#Primero convertir los datos a una x equivalente.
dummy2 = np.linspace(0,train_df.shape[0] - 1, train_df.shape[0]).reshape(-1,1)
reg = LinearRegression().fit(dummy2, train_df['FB'])
reg.score(dummy2, train_df['FB'])

In [ ]:
dummy3 = np.linspace(train_df.shape[0] - 1, trial_df.shape[0] - 1, test_df.shape[0]).reshape(-1,1)
test_results = reg.predict(dummy3)
reg.score(dummy3, test_df['FB'])

In [ ]:
plt.plot(dummy3, test_df['FB'], label = "Datos de prueba" )
plt.plot(dummy3, test_results, label = 'Modelo')
plt.title("Predicción de precios de la acción. ")
plt.ylabel("Precio")
plt.xlabel("Fecha")
plt.legend()
plt.savefig("img/Prediccion de precios.jpg")
plt.show()

In [ ]:
results2 = reg.predict(dummy)
plt.plot(train_df['date'],train_df['FB'], label = 'Datos de entrenamiento')
plt.plot(test_df['date'], test_df['FB'], label = 'Datos de prueba')
plt.plot(df['date'], results2, label = 'Modelo predictivo')
plt.title("Predicción de precios de la acción. ")
plt.ylabel("Precio")
plt.xlabel("Fecha")
plt.legend()
plt.savefig("img/Precio con prediccion.jpg")
plt.show()

¿Que podemos decir de todo esto?

- El precio de las acciones de Facebook muestra una tendencia a la alta.
- El crecimiento del precio a través de grandes periodos de tiempo sigue un comportamiento lineal.
- El modelo parece describir apropiadamente la tendencia.
- Las predicciones de los precios de las acciones tiene una varianza muy grande.

¿Cómo se puede mejorar? Se debe reducir el 'ruido'

### Opción 1: Detectar valores atípicos

A simple vista, se observa que existen valores dentro de la serie que se desvían mucho del comportamiento del modelo, se deben a eventos externos y son importantes de considerar cuando se trabaja con predicciones exactas del precio; sin embargo, provocan un ruido a tal grado que puede sesgar el modelo que busca predecir la tendencia.

Un **dato atípico** es uno que se aleja de forma extrema del comportamiento establecido de los datos (valores extremos).

Para detectarlos, se debe establecer un criterio. En este caso se puede considerar como valor extremo, un valor que se desvía significativamente de la tendencia lineal de los datos.

In [ ]:
#Ahora se quita la tendencia a los datos
var_analyze = df['FB'] - results2 #results 2 es la línea de la regresión
var_stand = pd.Series((var_analyze - var_analyze.mean())/(var_analyze.std())) #Se estandariza variable - promedio / desv std

In [ ]:
plt.plot(var_stand)
plt.title("Precio de acción sin tendencia.")
plt.savefig("Precio sin tendencia.jpg")
plt.savefig("img/Precio sin tendencia.jpg")
plt.show()

In [ ]:
threshold = 2.2 #2.2 desviaciones standar
df['normalized_open'] = var_stand
df['outliers'] = np.abs(var_stand) >= threshold
df_out = df[df['outliers'] == 1]

In [ ]:
plt.plot(df['date'], df['FB'])
plt.plot(df['date'], results2)
plt.scatter(df_out['date'], df_out['FB'], c = 'r')
plt.xlim([df['date'].min(), df['date'].max()])
plt.title('Valores extremos')
plt.xlabel('Fecha')
plt.ylabel('Precio de acción (USD)')
plt.savefig("img/valores extremos.jpg")
plt.show()

¿Se marcaron los valores atípicos?

In [ ]:
def remove_outliers(ts, outliers_idx):
    ts_clean = ts.copy()
    ts_clean.loc[outliers_idx] = np.nan
    ts_clean = ts_clean.interpolate(method="slinear")
    return ts_clean

In [ ]:
filt = remove_outliers(pd.Series(trial_df["FB"]), df_out.index)

In [ ]:
trial_df['filtered'] = filt

In [ ]:
plt.plot(df['date'], df['FB'], label = 'Datos originales')
plt.plot(df['date'], trial_df['filtered'], label = 'Datos sin anomalias')
plt.title("Predicción de precios de la acción. ")
plt.ylabel("Precio")
plt.xlabel("Fecha")
plt.legend()
plt.savefig("img/Precio sin valores extremos.jpg")
plt.show()

In [ ]:
test_sample = 350 #Declaro que voy a probar con 365 dias
train_df = trial_df.iloc[:(trial_df.shape[0] - test_sample),:] #Creo un set de entrenamiento de 0 hasta test_sample
test_df = trial_df.iloc[(trial_df.shape[0] - test_sample):,:] #creo un set de prueba con valores de test sample

In [ ]:
print(train_df.shape, test_df.shape) #Checo mis registros que estén bien

In [ ]:
#Primero convertir los datos a una x equivalente.
dummy2 = np.linspace(0,train_df.shape[0] - 1, train_df.shape[0]).reshape(-1,1)
reg = LinearRegression().fit(dummy2, train_df['filtered'])
reg.score(dummy2, train_df['filtered'])

In [ ]:
dummy3 = np.linspace(train_df.shape[0] - 1, trial_df.shape[0] - 1, test_df.shape[0]).reshape(-1,1)
test_results = reg.predict(dummy3)
reg.score(dummy3, test_df['filtered'])

In [ ]:
plt.plot(dummy3, test_df['filtered'], label = "Datos de prueba" )
plt.plot(dummy3, test_results, label = 'Modelo')
plt.title("Predicción de precios de la acción. ")
plt.ylabel("Precio")
plt.xlabel("Fecha")
plt.legend()
plt.savefig("img/Prediccion sin val extremos.jpg")
plt.show()

Es posible ver que quitar los valores extremos otorga una mejora global al modelo (R^2), sin embargo, todavía queda mucho trabajo por hacer si queremos extraer la tendencia. Por lo tanto, necesitamos una forma de reducir los efectos del ruido y de la estacionalidad para tener un mejor resultado.

### Opción 2: Quitar el ruido y mejorar la regresión lineal

Como se menciona anteriormente, la presencia de *ruido* y *estacionalidad* en los datos puede dificultar la extracción de la verdadera tendencia, además, puede resultar en que un buen modelo termine por ser evaluado de una forma negativa. ¿Que se puede hacer para reducir el ruido dentro de la variable de interés?

Usar el promedio: Es una cantidad que busca representar números variados con un solo valor; sin embargo, no se debe confundir con el cálculo del nivel, ya que ese valor es constante a través de la serie de tiempo, mientras que este promedio si varía con respecto al tiempo.

Combinando el valor actual con los valores pasados de la serie de tiempo, se puede llegar a reconstruir la tendencia, mientras que el ruido desaparece debido al promedio.

Hay diversas formas de realizar este filtrado: El promedio móvil (Simple Moving Average (SMA)) calcula el promedio aritmético de unos pocos valores previos para estimar el nuevo punto que es calculado con la siguiente función.

\begin{align}
\mathrm{MA}_n = \frac{x_n + x_{n-1} + x_{n-2} + x_{n-3} ..... + x_{n-m+1}}{m}
\end{align}

Una ventaja de esta función es que debido a que los valores futuros se calculan con la misma fórmula, se puede utilizar el promedio previo para calcular el valor futuro de la función.

\begin{align}
\mathrm{MA}_{n+1} = \frac{x_{n+1}}{m} + \frac{x_n + x_{n-1} + x_{n-2} + x_{n-3} + .... x_{n-m+2}}{m}\\
\mathrm{MA}_{n+1} = \frac{x_{n+1} - x_{n-m+1}}{m} + \mathrm{MA}_n
\end{align}

Por ejemplo, se tienen como valores de la serie de tiempo [0,-0.1, 0.9, 1.5, 1.2, 2.1, 2.3, 2, 2.9, 3], por ahora se asume que la separación entre los valores es la misma.

¿Como se realiza el cálculo del SMA?

\begin{align}
\mathrm{MA}_1& = \frac{-0.1 + 0.0}{2} = -0.05 \\
\mathrm{MA}_2& = \frac{0.9 - 0.0}{2} - 0.05 = 0.40 \\
\mathrm{MA}_3& = \frac{1.5 + 0.1}{2} + 0.40 = 1.20 \\
\mathrm{MA}_4& = \frac{1.2 - 0.9}{2} + 1.20 = 1.35 \\
\mathrm{MA}_5& = \frac{2.1 - 1.5}{2} + 1.35 = 1.65 \\
\mathrm{MA}_6& = \frac{2.3 - 1.2}{2} + 1.65 = 2.20 \\
\mathrm{MA}_7& = \frac{2.0 - 2.1}{2} + 2.20 = 2.15 \\
\mathrm{MA}_8& = \frac{2.9 - 2.3}{2} + 2.15 = 2.45 \\
\mathrm{MA}_9& = \frac{3.0 - 2.0}{2} + 2.45 = 2.95 
\end{align}

In [ ]:
# ¿Que efecto tiene este rolling mean sobre la forma de la data?

my_data = pd.Series([0,-0.1, 0.9, 1.5, 1.2, 2.1, 2.3, 2, 2.9, 3])
plt.plot(list(range(len(my_data))), my_data , label = 'Datos')
plt.plot(list(range(len(my_data))), my_data.rolling(4, center = True).mean(), label = 'SMA') #¿Qué pasa si tomo mas datos?
plt.legend()
plt.savefig("img/Precio con promedio movil.jpg")
plt.show()

¿Por qué la línea naranja es más corta?

In [ ]:
print(my_data)

In [ ]:
#Como calcular el rolling mean
print(my_data.rolling(2, center = True).mean())

In [ ]:
#Ahora... como se verá con la data de precios de acciones.
df['FB_cl'] = df.iloc[:,3] - df.iloc[:, 3].mean()

In [ ]:
#Calculamos el rolling mean
close_4 = df['FB'].rolling(4, center = True).mean()

In [ ]:
#y ahora a visualizar
fig = plt.figure(figsize=(15,10)) 
ax = fig.add_subplot(1, 1, 1)
ax.plot(df.date, df['FB'], linewidth = 2, c = 'blue', label = 'Datos originales')
ax.plot(df.date, close_4, linewidth = 2, c = 'red', label = 'SMA_4')
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_title('Trend', fontsize = 18)
ax.set_xlabel('Time ', fontsize = 17)
ax.set_ylabel('Stock Price', fontsize = 17)
plt.legend()
plt.savefig("img/Precio con promedio movil c.jpg")
plt.show()

In [ ]:
#Que pasa si aumentamos el número de valores a utilizar?
close_8 = df['FB'].rolling(8, center = True).mean()
close_12 = df['FB'].rolling(12, center = True).mean()
close_24 = df['FB'].rolling(24, center = True).mean()

In [ ]:
fig = plt.figure() 
ax = fig.add_subplot(1, 1, 1)
ax.plot(df.date, df['FB'], linewidth = 2, label = 'Datos originales')
#ax.plot(df.date, close_8, linewidth = 2, c = 'red', label = 'SMA_8')
#ax.plot(df.date, close_12, linewidth = 2, c = 'blue', label = 'SMA_12')
ax.plot(df.date, close_24, linewidth = 2, c = 'red', label = 'SMA_24')
ax.tick_params(axis='both', which='major')
ax.set_title('Trend')
ax.set_xlabel('Time ')
ax.set_ylabel('Stock Price')
plt.legend()
plt.savefig("img/Precio con promedio movil 24.jpg")
plt.show()

Entre más valores se utilizan para el SMA, más se reduce el ruido, pero la señal se ve atrasada y tiene un sesgo hacia precios bajos. Esto se debe a que entre más valores se utilizan para calcular el promedio, más se aleja del valor actual y se crea un gran sesgo hacia el pasado. Esto se puede resolver centrando el SMA.

In [ ]:
print(trial_df)

In [ ]:
trial_df_roll = trial_df.copy()
trial_df_roll['Rolling'] = trial_df_roll['FB'].rolling(36, center = True).mean()
trial_df_roll.head(15)

In [ ]:
trial_df_roll.dropna(axis = 0, inplace = True)

In [ ]:
test_sample = 100
train_df = trial_df_roll.iloc[:(trial_df_roll.shape[0] - test_sample),:]
test_df = trial_df_roll.iloc[(trial_df_roll.shape[0] - test_sample):,:]

In [ ]:
print(train_df.shape, test_df.shape)

In [ ]:
#Primero tenemos que convertir los datos a una x equivalente.
dummy = np.linspace(0,trial_df_roll.shape[0] - 1, trial_df_roll.shape[0]).reshape(-1,1)
dummy2 = np.linspace(0,train_df.shape[0] - 1, train_df.shape[0]).reshape(-1,1)
reg = LinearRegression().fit(dummy2, train_df['Rolling'])

In [ ]:
dummy3 = np.linspace(train_df.shape[0] - 1, trial_df.shape[0] - 1, test_df.shape[0]).reshape(-1,1)
test_results = reg.predict(dummy3)
print('El valor de R2 es: {}'.format(reg.score(dummy3, test_df['Rolling'])))

In [ ]:
plt.plot(dummy3, test_df['Rolling'], label = "Set de Prueba")
plt.plot(dummy3, test_results, label = "Modelo")
plt.title("Predicción de precios de la acción. ")
plt.ylabel("Precio")
plt.xlabel("Fecha")
plt.legend()
plt.show()
plt.show()

In [ ]:
results2 = reg.predict(dummy)
plt.plot(train_df['date'],train_df['Rolling'], label = "Set de Entrenamiento")
plt.plot(test_df['date'], test_df['Rolling'], label = "Set de Prueba")
plt.plot(trial_df_roll['date'], results2, label = "Modelo")
plt.title("Predicción de precios de la acción. ")
plt.ylabel("Precio")
plt.xlabel("Fecha")
plt.legend()
plt.savefig("Precio_RL.jpg")
plt.show()
plt.show()

El SMA logra quitar las oscilaciones chicas de la señal, pero los valores extremos siguen siendo demasiado para ello.

### Opción 3: Combinar las 2 anteriores

In [ ]:
trial_df_combined = trial_df.copy()
trial_df_combined['combined'] = trial_df_combined['filtered'].rolling(4, center = True).mean()
trial_df_combined.head()

In [ ]:
trial_df_combined.dropna(axis = 0, inplace = True)

In [ ]:
test_sample = 350
train_df = trial_df_combined.iloc[:(trial_df_combined.shape[0] - test_sample),:]
test_df = trial_df_combined.iloc[(trial_df_combined.shape[0] - test_sample):,:]

In [ ]:
print(train_df.shape, test_df.shape)

In [ ]:
#Primero tenemos que convertir los datos a una x equivalente.
dummy = np.linspace(0,trial_df_combined.shape[0] - 1, trial_df_combined.shape[0]).reshape(-1,1)
dummy2 = np.linspace(0,train_df.shape[0] - 1, train_df.shape[0]).reshape(-1,1)
reg = LinearRegression().fit(dummy2, train_df['combined'])

In [ ]:
dummy3 = np.linspace(train_df.shape[0] - 1, trial_df.shape[0] - 1, test_df.shape[0]).reshape(-1,1)
test_results = reg.predict(dummy3)
reg.score(dummy3, test_df['combined'])

In [ ]:
print(dummy3.shape, test_df['combined'].shape)

In [ ]:
plt.plot(dummy3, test_df['combined'])
plt.plot(dummy3, test_results)
plt.show()